# Clase Supervised Classification ML

## Importar librerías

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from joblib import dump, load
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

## Importar los datos

In [2]:
# Tu código aquí debajo
df = pd.read_excel('diabetes_clean.xlsx')
df.sample(8)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Satisfaction,Job,Outcome
92,7,160,54,32,175,30.5,588.0,39,Satisfied,High School Teacher,1
612,10,101,76,48,180,32.9,171.0,63,Unsatisfied,High School Teacher,0
236,1,128,48,45,194,40.5,613.0,2,Extremely unsatisfied,High School Teacher,1
222,0,104,64,23,116,27.8,454.0,23,Extremely unsatisfied,High School Teacher,0
46,0,105,64,41,142,41.5,173.0,22,Unsatisfied,Financial Analyst,0
559,4,127,88,11,155,34.5,598.0,28,Extremely satisfied,Human Resources,0
230,7,136,74,26,135,26.0,647.0,51,Extremely satisfied,High School Teacher,0
289,5,158,70,0,0,29.8,207.0,63,Unsatisfied,Human Resources,0


## Detectar y tratar duplicados

In [3]:
# Tu código aquí debajo
df.duplicated().sum()

0

## Detectar y tratar valores nulos

In [4]:
# Tu código aquí debajo
df.isnull().sum()


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Satisfaction                0
Job                         0
Outcome                     0
dtype: int64

In [5]:
# Tu código aquí debajo



## Detectar y tratar valores atípicos (ouliers)

In [6]:
# Tu código aquí debajo
numericas=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']

# FUNCIÓN QUE DEVUELVE LOS NÚMEROS DE FILA EN LOS QUE LA VARIABLE EN CUESTIÓN ES OULIER
def outliers(var):
  q1=var.quantile(0.25)
  q3=var.quantile(0.75)
  riq=q3-q1
  sup=q3+1.5*(riq)
  inf=q1-1.5*(riq)
  outl=(var>sup) | (var<inf)
  return outl

for i in df.loc[:,numericas]:
  print("Cantida de oultliers de la variable",i,":",outliers(df[i]).sum())
# Eliminar outliers
# Tu código
for i in df.loc[:,numericas]:
  df=df[~outliers(df[i])]
df.sample(5)

Cantida de oultliers de la variable Pregnancies : 0
Cantida de oultliers de la variable Glucose : 9
Cantida de oultliers de la variable BloodPressure : 4
Cantida de oultliers de la variable SkinThickness : 0
Cantida de oultliers de la variable Insulin : 4
Cantida de oultliers de la variable BMI : 4
Cantida de oultliers de la variable DiabetesPedigreeFunction : 10
Cantida de oultliers de la variable Age : 10


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Satisfaction,Job,Outcome
14,3,126,88,41,235,39.3,704.0,27,Extremely unsatisfied,Human Resources,0
527,2,120,54,0,0,26.8,455.0,27,Extremely satisfied,Physicians,0
565,9,164,78,0,0,32.8,148.0,45,Extremely unsatisfied,Human Resources,1
495,6,117,96,0,0,28.7,157.0,30,Satisfied,High School Teacher,0
67,2,110,74,29,125,32.4,698.0,27,Extremely unsatisfied,High School Teacher,0


# Preparamos los datos



## Transformar variables ordinales en numéricas

Como vemos tenemos la variable Satisfaction que es una variable ordinal y podemos suponer que el impacto que podrá tener sobre la variable a predecir (diabetes si/no) permanecerá constante al pasar de una categoría a la siguiente. por eso, la codificaremos como variable ordinal

In [7]:
# Creamos el objeto que realizará la transformación dándole el orden en una lista con doble corchete
enc=OrdinalEncoder(categories=[['Extremely unsatisfied','Unsatisfied','Satisfied','Extremely satisfied']],dtype='int')
# Particularizamos el codificador en nuestros datos
# Tu código
enc.fit(df[['Satisfaction']])
# Aplicamos el codificador ya particularizado a nuestros datos y lo guardamos machacando la misma variable
# Tu código
df['Satisfaction']=enc.transform(df[['Satisfaction']])
print(df.shape)
df.sample(5)


(576, 11)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Satisfaction,Job,Outcome
463,1,119,44,47,63,35.5,0.28,25,1,Financial Analyst,0
493,6,105,80,28,0,32.5,878.00,26,0,High School Teacher,0
425,0,117,66,31,188,30.8,493.00,22,0,Human Resources,0
15,8,99,84,0,0,35.4,388.00,50,0,Human Resources,0
486,1,124,60,32,0,35.8,514.00,21,3,High School Teacher,0


## Transformar variables nominales en numéricas

Como vemos tenemos la variable Job que es una variable nominal (sin orden) y debenmos codificarla mediante el One-Hot encoder

In [8]:
# Tu código
# Creamos el objeto que realizará la transformación

#Instancio el OneHot
onehot=OneHotEncoder()

# Aplico el OneHot a la columna job y guardo el resultao en a
a=onehot.fit_transform(df[['Job']])

# Convierto el a en data frame y lo llamo encoded_df
encoded_df = pd.DataFrame(a.toarray(), columns=onehot.get_feature_names_out(['Job']))

# Reseteo el índice de los dos data frames antes de concatenarlos
df.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)

# concateno los dos data frames y los guardo machacando el df
df=pd.concat([df,encoded_df],axis=1)

# Elimino la columna Job original antigua, la categórica
df.drop(columns='Job',inplace=True)

# Muestro tamaño yu sample del data frame df transformado
print(df.shape)
df.sample(5)



(576, 20)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Satisfaction,Outcome,Job_Database Administrator,Job_Epidemiologist,Job_Financial Analyst,Job_Firefighter,Job_High School Teacher,Job_Human Resources,Job_Legislator,Job_Market Research Analyst,Job_Physicians,Job_Statistician
366,0,135,68,42,250,42.3,365.00,24,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
94,6,111,64,39,0,34.2,0.26,24,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
204,2,108,62,32,56,25.2,128.00,21,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
148,8,85,55,20,0,24.4,136.00,42,2,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
426,2,95,54,14,88,26.1,748.00,22,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## 0. Separar la variable a predecir y las predictoras

In [9]:
X=df.drop(columns=['Outcome'],inplace=False)
y=df['Outcome']

## 1. CROSS-VALIDATION

In [10]:
# Tu código aquí debajo
from sklearn.model_selection import KFold

In [11]:
# Cojemos pocos datos
datos_p = df.sample(10)
datos_p

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Satisfaction,Outcome,Job_Database Administrator,Job_Epidemiologist,Job_Financial Analyst,Job_Firefighter,Job_High School Teacher,Job_Human Resources,Job_Legislator,Job_Market Research Analyst,Job_Physicians,Job_Statistician
71,7,81,78,40,48,46.7,261.0,42,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
142,4,141,74,0,0,27.6,244.0,40,2,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
523,2,122,76,27,200,35.9,483.0,26,1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
109,5,106,82,30,0,39.5,286.0,38,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
341,2,134,70,0,0,28.9,542.0,23,3,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
357,0,119,66,27,0,38.8,259.0,22,2,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
139,6,87,80,0,0,23.2,84.0,32,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
145,3,111,62,0,0,22.6,142.0,21,3,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
197,3,111,56,39,0,30.1,557.0,30,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
277,3,81,86,16,66,27.5,306.0,22,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [12]:
my_kfold = KFold(n_splits = 30)

In [13]:
for i_train, i_test in my_kfold.split(df):
    print("TRAIN:", i_train, "TEST:", i_test)

TRAIN: [ 20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109
 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127
 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163
 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181
 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199
 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217
 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235
 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253
 254 255 256 257 258 259 260 261 262 263 264

## Evaluar modelos

### 1.1 Logistic Regression

In [14]:
estandarizador = StandardScaler()
LR = LogisticRegression()
accuracis = []
precisions = []
recalls = []
f1s = []

for i_train, i_test in my_kfold.split(df):
    X_train, X_test = X.iloc[i_train], X.iloc[i_test]
    y_train, y_test = y.iloc[i_train], y.iloc[i_test]
    estandarizador.fit(X_train)
    X_train_std=estandarizador.transform(X_train)
    X_test_std=estandarizador.transform(X_test)
    
    LR.fit(X_train_std,y_train)
    predictions = LR.predict(X_test_std)
    
    accuracis.append(accuracy_score(y_test, predictions))
    precisions.append(precision_score(y_test, predictions))
    recalls.append(recall_score(y_test, predictions))
    f1s.append(f1_score(y_test, predictions))

c:\Users\GuillemEdoBertran\.conda\envs\ML_IMPELIA\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
metricas_LR=[]
metricas_LR.append(sum(accuracis)/len(accuracis))
metricas_LR.append(sum(precisions)/len(precisions))
metricas_LR.append(sum(recalls)/len(recalls))
metricas_LR.append(sum(f1s)/len(f1s))
metricas_LR

[0.7918421052631577,
 0.6283333333333333,
 0.43323472823472825,
 0.4933620790973732]

### 1.2 Decision Tree

In [16]:
DT = DecisionTreeClassifier()
accuracis = []
precisions = []
recalls = []
f1s = []

for i_train, i_test in my_kfold.split(df):
    X_train, X_test = X.iloc[i_train], X.iloc[i_test]
    y_train, y_test = y.iloc[i_train], y.iloc[i_test]
    
    DT.fit(X_train,y_train)
    predictions = DT.predict(X_test)
    
    accuracis.append(accuracy_score(y_test, predictions))
    precisions.append(precision_score(y_test, predictions))
    recalls.append(recall_score(y_test, predictions))
    f1s.append(f1_score(y_test, predictions))

In [17]:
metricas_DT=[]
metricas_DT.append(sum(accuracis)/len(accuracis))
metricas_DT.append(sum(precisions)/len(precisions))
metricas_DT.append(sum(recalls)/len(recalls))
metricas_DT.append(sum(f1s)/len(f1s))
metricas_DT

[0.7078947368421052, 0.47246632996633, 0.45616883116883117, 0.4475866943514003]

### 1.3 Random Forest

In [18]:
estandarizador = StandardScaler()
RF = LogisticRegression()
accuracis = []
precisions = []
recalls = []
f1s = []

for i_train, i_test in my_kfold.split(df):
    X_train, X_test = X.iloc[i_train], X.iloc[i_test]
    y_train, y_test = y.iloc[i_train], y.iloc[i_test]
    estandarizador.fit(X_train)
    X_train_std=estandarizador.transform(X_train)
    X_test_std=estandarizador.transform(X_test)
    
    RF.fit(X_train_std,y_train)
    predictions = RF.predict(X_test_std)
    
    accuracis.append(accuracy_score(y_test, predictions))
    precisions.append(precision_score(y_test, predictions))
    recalls.append(recall_score(y_test, predictions))
    f1s.append(f1_score(y_test, predictions))

c:\Users\GuillemEdoBertran\.conda\envs\ML_IMPELIA\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
metricas_RF=[]
metricas_RF.append(sum(accuracis)/len(accuracis))
metricas_RF.append(sum(precisions)/len(precisions))
metricas_RF.append(sum(recalls)/len(recalls))
metricas_RF.append(sum(f1s)/len(f1s))
metricas_RF

[0.7918421052631577,
 0.6283333333333333,
 0.43323472823472825,
 0.4933620790973732]

In [20]:
dict={"Logistic Regression":metricas_LR, "DecisionTree":metricas_DT, "RandomForest":metricas_RF}
Resultados=pd.DataFrame(dict,index=["Acuracy","Precision","Recall","F1"])
Resultados

,Logistic Regression,DecisionTree,RandomForest
Acuracy,0.791842,0.707895,0.791842
Precision,0.628333,0.472466,0.628333
Recall,0.433235,0.456169,0.433235
F1,0.493362,0.447587,0.493362
